https://moncoachdata.com/blog/groupby-de-pandas/

In [388]:
import pandas as pd

df = pd.DataFrame({
   "nom": ["John", "Lydia", "Laury", "Arnaud", "Enzo"],
   "genre": ["Homme", "Femme", "Femme", "Homme", "Homme"],
   "taille": [182, 166, 170, 178, 172],
   "education": ["Master", "Master", "Licence", "Licence", "Master"],
   "salaire": [65000, 72000, 74000, 68000, 80000]
})
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,Homme,172,Master,80000


In [389]:
# df.groupby("genre").mean() -> error

À l’avenir, il faudra spécifier à GroupBy de Pandas de spécifier uniquement les colonnes numériques ou sélectionner manuellement les colonnes (numériques) qui nous intéressent !

In [390]:
# groupby(critère) puis sélection de colonnes avant aggrégation
df.groupby("genre")[['taille', 'salaire']].mean()

,taille,salaire
genre,,
Femme,168.000000,73000.0
Homme,177.333333,71000.0


Dans certains cas, nous ne voulons appliquer la fonction d’agrégation qu’à une colonne spécifique. Une façon de procéder consiste à filtrer les colonnes avant d’appliquer la fonction groupby.

In [391]:
# sélection de colonnes puis groupby(critère)
df[["genre","salaire","taille"]].groupby("genre").mean()

,salaire,taille
genre,,
Femme,73000.0,168.000000
Homme,71000.0,177.333333


Ce que nous avons fait dans la deuxième étape n’est pas la solution optimale. Un ensemble de données typique de la vie réelle contient plusieurs colonnes et nous pouvons avoir besoin de calculer des agrégations non pas sur toutes les colonnes, mais sur un grand nombre d’entre elles.

Dans ce cas, le filtrage des colonnes est une tâche fastidieuse. Une meilleure méthode consiste à utiliser la fonction NamedAgg de Pandas.

In [392]:
# aggregate (column, method)
df.groupby("genre").agg(salaire_moyen = pd.NamedAgg("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,71000.0


Il existe même une méthode plus simple que celle de la troisième étape. La syntaxe est la suivante :

In [393]:
# or simply
df.groupby("genre").agg(salaire_moyen=("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,71000.0


Une fois les groupes générés, nous pouvons calculer autant d’agrégations que nécessaire. Par exemple, le code suivant calcule le salaire médian et la taille moyenne pour chaque sexe.

In [394]:
df.groupby("genre").agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

,salaire_median,taille_moyenne
genre,,
Femme,73000.0,168.000000
Homme,68000.0,177.333333


Jusqu’à présent, les groupes sont affichés en tant qu’index d’un DataFrame. Dans certains cas, il est préférable de les afficher sous forme de colonne dans le DataFrame. Nous pouvons y parvenir en utilisant le paramètre as_index.


In [395]:
df.groupby("genre", as_index=False).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

,genre,salaire_median,taille_moyenne
0,Femme,73000.0,168.000000
1,Homme,68000.0,177.333333


Tout comme nous pouvons calculer des agrégations sur plusieurs colonnes, nous pouvons créer des groupes basés sur plusieurs colonnes. Si nous transmettons deux colonnes à la fonction groupby, celle-ci crée des groupes basés sur la combinaison de valeurs distinctes dans chaque colonne.

groupby([ordered list of criters to group by levels])

In [396]:
df.groupby(["genre","education"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

salaire_median  taille_moyenne
genre education                                
Femme Licence           74000.0           170.0
      Master            72000.0           166.0
Homme Licence           68000.0           178.0
      Master            72500.0           177.0

Order matters !

In [397]:
df.groupby(["education", "genre"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
)

salaire_median  taille_moyenne
education genre                                
Licence   Femme         74000.0           170.0
          Homme         68000.0           178.0
Master    Femme         72000.0           166.0
          Homme         72500.0           177.0

Ce n’est pas nécessaire dans notre cas, mais lorsque nous avons plusieurs groupes, nous pouvons vouloir trier les groupes sur la base des valeurs agrégées. Supposons que nous ayons 100 colonnes et que nous devions trouver les 3 premiers groupes en termes de salaire moyen le plus élevé. Dans ce cas, il est nécessaire de trier les résultats.

La fonction sort_values peut être utilisée avec la fonction groupby comme suit :

In [398]:
df.groupby(["genre","education"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
).sort_values(by="salaire_median", ascending=False)

,,salaire_median,taille_moyenne
genre,education,,
Femme,Licence,74000.0,170.0
Homme,Master,72500.0,177.0
Femme,Master,72000.0,166.0
Homme,Licence,68000.0,178.0


In [399]:
df.groupby(["education", "genre"], as_index=True).agg(
   salaire_median=("salaire","median"),
   taille_moyenne=("taille","mean")
).sort_values(by="salaire_median", ascending=False)

salaire_median  taille_moyenne
education genre                                
Licence   Femme         74000.0           170.0
Master    Homme         72500.0           177.0
          Femme         72000.0           166.0
Licence   Homme         68000.0           178.0

Notre base de données ne contient pas de valeurs manquantes. Toutefois, les ensembles de données réels sont susceptibles d’en contenir. S’il y a des valeurs manquantes dans la colonne utilisée pour le regroupement, ces lignes sont supprimées par défaut.


In [400]:
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,Homme,172,Master,80000


Commençons par mettre à jour une valeur manquante dans notre base de données.

In [401]:
df.iloc[4,1] = None
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,None,172,Master,80000


In [402]:
df.groupby("genre").agg(salaire_moyen=("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,66500.0


Comme tu peux le constater, la valeur manquante (None) dans la colonne ‘genre’ est ignorée. Cependant, il est important de prendre en compte les valeurs manquantes dans de nombreux cas. Nous pouvons définir le paramètre dropna sur False pour inclure les valeurs manquantes dans les résultats.

In [403]:
df.groupby("genre", dropna=False).agg(salaire_moyen=("salaire","mean"))

,salaire_moyen
genre,
Femme,73000.0
Homme,66500.0
NaN,80000.0


Nous avons vu comment calculer plusieurs agrégations à l’étape 5. Une autre méthode pour cette opération consiste à passer une liste d’agrégations à la fonction agg

In [404]:
df[["genre", 'taille', 'salaire']].groupby("genre", dropna=False).agg(["mean","count"])

taille        salaire      
        mean count     mean count
genre                            
Femme  168.0     2  73000.0     2
Homme  180.0     2  66500.0     2
NaN    172.0     1  80000.0     1

https://pandas.pydata.org/docs/user_guide/groupby.html

In [405]:
import numpy as np
speeds = pd.DataFrame(
   [
   ("bird", "Falconiformes", 389.0),
   ("bird", "Psittaciformes", 24.0),
   ("mammal", "Carnivora", 80.2),
   ("mammal", "Primates", np.nan),
   ("mammal", "Carnivora", 58),
   ],
   index=["falcon", "parrot", "lion", "monkey", "leopard"],
   columns=("class", "order", "max_speed"),
   )
    

speeds


,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [406]:
grouped = speeds.groupby("class")
grouped["max_speed"].mean()

class
bird      206.5
mammal     69.1
Name: max_speed, dtype: float64

In [407]:
grouped = speeds.groupby(["class", "order"])
grouped["max_speed"].mean()


class   order         
bird    Falconiformes     389.0
        Psittaciformes     24.0
mammal  Carnivora          69.1
        Primates            NaN
Name: max_speed, dtype: float64

In [408]:
df1 = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)


df1

,A,B,C,D
0,foo,one,-1.320193,-1.229231
1,bar,one,-0.343423,0.612309
2,foo,two,-0.723008,-1.489201
3,bar,three,1.038926,-0.671137
4,foo,two,0.837079,1.260185
5,bar,two,-0.066047,0.423842
6,foo,one,1.292443,1.962022
7,foo,three,0.240894,-0.853801


In [409]:
groupedA_df1 = df1.groupby("A")
groupedA_df1[["C","D"]].mean()

,C,D
A,,
bar,0.209819,0.121671
foo,0.065443,-0.070005


In [410]:
groupedB_df1 = df1.groupby("B")
groupedB_df1[["C","D"]].mean()


,C,D
B,,
one,-0.123725,0.448367
three,0.639910,-0.762469
two,0.016008,0.064942


In [411]:
groupedAB_df1 = df1.groupby(["A", "B"])
groupedAB_df1[["C","D"]].mean()

C         D
A   B                        
bar one   -0.343423  0.612309
    three  1.038926 -0.671137
    two   -0.066047  0.423842
foo one   -0.013875  0.366395
    three  0.240894 -0.853801
    two    0.057035 -0.114508

In [412]:
df1_indexAB = df1.set_index(["A", "B"])
df1_indexAB

C         D
A   B                        
foo one   -1.320193 -1.229231
bar one   -0.343423  0.612309
foo two   -0.723008 -1.489201
bar three  1.038926 -0.671137
foo two    0.837079  1.260185
bar two   -0.066047  0.423842
foo one    1.292443  1.962022
    three  0.240894 -0.853801

If we also have a MultiIndex on columns A and B, we can group by all the columns except the one we specify:

In [413]:
# only A
groupedAB = df1_indexAB.groupby(level=df1_indexAB.index.names.difference(["B"]))
groupedAB[["C","D"]].sum()


,C,D
A,,
bar,0.629456,0.365014
foo,0.327214,-0.350026


In [414]:
groupedAB.sum()

,C,D
A,,
bar,0.629456,0.365014
foo,0.327214,-0.350026


In [415]:
# only B from multiindex dataframe
grouped = df1_indexAB.groupby(level=df1_indexAB.index.names.difference(["A"]))
grouped.mean()  # aggregate

,C,D
B,,
one,-0.123725,0.448367
three,0.639910,-0.762469
two,0.016008,0.064942


In [416]:
df

,nom,genre,taille,education,salaire
0,John,Homme,182,Master,65000
1,Lydia,Femme,166,Master,72000
2,Laury,Femme,170,Licence,74000
3,Arnaud,Homme,178,Licence,68000
4,Enzo,None,172,Master,80000


The above GroupBy will split the DataFrame on its index (rows). To split by columns, first do a transpose:

In [417]:
dfT = df1.T

In [418]:
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'


grouped = df1.T.groupby(get_letter_type)
list(grouped)

[('consonant',
            0         1         2         3         4         5         6  \
  B       one       one       two     three       two       two       one   
  C -1.320193 -0.343423 -0.723008  1.038926  0.837079 -0.066047  1.292443   
  D -1.229231  0.612309 -1.489201 -0.671137  1.260185  0.423842  1.962022   
  
            7  
  B     three  
  C  0.240894  
  D -0.853801  ),
 ('vowel',
       0    1    2    3    4    5    6    7
  A  foo  bar  foo  bar  foo  bar  foo  foo)]

In [419]:
#df.T.reset_index()
# Duplicate the index into a new column
dfT['Index_Column'] = dfT.index
dfT['Index_Column'] = dfT['Index_Column'].apply(get_letter_type)
dfT

,0,1,2,3,4,5,6,7,Index_Column
A,foo,bar,foo,bar,foo,bar,foo,foo,vowel
B,one,one,two,three,two,two,one,three,consonant
C,-1.320193,-0.343423,-0.723008,1.038926,0.837079,-0.066047,1.292443,0.240894,consonant
D,-1.229231,0.612309,-1.489201,-0.671137,1.260185,0.423842,1.962022,-0.853801,consonant


In [420]:
dfT.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, A to D
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   0             4 non-null      object
 1   1             4 non-null      object
 2   2             4 non-null      object
 3   3             4 non-null      object
 4   4             4 non-null      object
 5   5             4 non-null      object
 6   6             4 non-null      object
 7   7             4 non-null      object
 8   Index_Column  4 non-null      object
dtypes: object(9)
memory usage: 492.0+ bytes


In [421]:
dfT = dfT.set_index("Index_Column")
dfT

,0,1,2,3,4,5,6,7
Index_Column,,,,,,,,
vowel,foo,bar,foo,bar,foo,bar,foo,foo
consonant,one,one,two,three,two,two,one,three
consonant,-1.320193,-0.343423,-0.723008,1.038926,0.837079,-0.066047,1.292443,0.240894
consonant,-1.229231,0.612309,-1.489201,-0.671137,1.260185,0.423842,1.962022,-0.853801


In [422]:
#grouped = dfT.groupby(['Index_Column']) 


In [423]:
grouped[list(dfT.columns[[0]])].obj

,0,1,2,3,4,5,6,7
A,foo,bar,foo,bar,foo,bar,foo,foo
B,one,one,two,three,two,two,one,three
C,-1.320193,-0.343423,-0.723008,1.038926,0.837079,-0.066047,1.292443,0.240894
D,-1.229231,0.612309,-1.489201,-0.671137,1.260185,0.423842,1.962022,-0.853801


In [424]:
grouped.obj.columns

RangeIndex(start=0, stop=8, step=1)

In [425]:
grouped[grouped.obj.columns]

In [426]:
df2 = df.set_index(list(df.columns[[0]]))
df2

,genre,taille,education,salaire
nom,,,,
John,Homme,182,Master,65000
Lydia,Femme,166,Master,72000
Laury,Femme,170,Licence,74000
Arnaud,Homme,178,Licence,68000
Enzo,None,172,Master,80000


In [427]:
list(grouped)

[('consonant',
            0         1         2         3         4         5         6  \
  B       one       one       two     three       two       two       one   
  C -1.320193 -0.343423 -0.723008  1.038926  0.837079 -0.066047  1.292443   
  D -1.229231  0.612309 -1.489201 -0.671137  1.260185  0.423842  1.962022   
  
            7  
  B     three  
  C  0.240894  
  D -0.853801  ),
 ('vowel',
       0    1    2    3    4    5    6    7
  A  foo  bar  foo  bar  foo  bar  foo  foo)]

In [428]:
#grouped.loc["consonant"]

In [429]:
# Sample DataFrame
data = {
    'Category': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Values1': [1, 2, 3, 4, 5, 6],
    'Values2': [10, 20, 30, 40, 50, 60]
}

df = pd.DataFrame(data)
print("Original DataFrame:")
print(df)

Original DataFrame:
  Category  Values1  Values2
0        A        1       10
1        A        2       20
2        B        3       30
3        B        4       40
4        C        5       50
5        C        6       60


In [430]:
# Grouping the DataFrame by the 'Category' column
grouped = df.groupby('Category')
grouped


In [431]:

# Accessing columns from the grouped object
columns = grouped.obj.columns
print("\nColumns in the grouped object:")
print(columns)


Columns in the grouped object:
Index(['Category', 'Values1', 'Values2'], dtype='object')


In [432]:
# Aggregating the grouped data (summing the values)
aggregated = grouped.sum()
print("\nAggregated DataFrame:")
print(aggregated)

# Accessing columns of the aggregated DataFrame
aggregated_columns = aggregated.columns
print("\nColumns in the aggregated DataFrame:")
print(aggregated_columns)


Aggregated DataFrame:
          Values1  Values2
Category                  
A               3       30
B               7       70
C              11      110

Columns in the aggregated DataFrame:
Index(['Values1', 'Values2'], dtype='object')
